# Predicting Conversations Gone Awry With Convokit

This interactive tutorial demonstrates how to predict whether a conversation will eventually lead to a personal attack, as seen in the paper [Conversations Gone Awry: Detecting Early Signs of Conversational Failure](http://www.cs.cornell.edu/~cristian/Conversations_gone_awry.html), using the tools provided by convokit. It also serves as an illustration of how to use two of convokit's main features: question typology and politeness strategies.

In [1]:
import os
import pkg_resources
import json
import itertools
import spacy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut
from sklearn.feature_selection import f_classif, SelectPercentile
from scipy import stats
from collections import defaultdict
from functools import partial
from multiprocessing import Pool
from itertools import combinations

from convokit import Corpus, QuestionTypology, download, MotifsExtractor, QuestionTypologyUtils, PolitenessStrategies


## Step 1: Load the corpus

In [2]:
# Download the Conversations Gone Awry corpus, provided as part of convokit
if not os.path.exists(os.path.join(pkg_resources.resource_filename("convokit", ""), "downloads", "conversations-gone-awry-corpus")):
    download("conversations-gone-awry-corpus")

In [3]:
# construct a convokit Corpus object from the downloaded data. The Corpus class provides functionality for
# convenient manipulation of text corpora.
awry_corpus = Corpus(filename=os.path.join(pkg_resources.resource_filename("convokit", ""), "downloads", "conversations-gone-awry-corpus"))

## Step 2: Extract prompt types features

In this step, we will extract the first of the two types of pragmatic features seen in the paper: prompt types. We can learn prompt types and compute types for each utterance in the corpus using convokit's QuestionTypology class.

In [4]:
# we will train the QuestionTypology object on convokit's wiki corpus. Let's first download the corpus...
if not os.path.exists(os.path.join(pkg_resources.resource_filename("convokit", ""), "downloads", "wiki-corpus")):
    download("wiki-corpus")

We now train a QuestionTypology object on the downloaded wiki corpus

In [5]:
# some parameters to get us started. Note that "dataset name" refers not to the name of the *source* dataset,
# but the name we want to give to trained output files of the QuestionTypology object. We could have given
# any name we liked - we chose "wiki-corpus-awry" to keep things clear.
dataset_name = "wiki-corpus-awry"
num_clusters = 6

# load the wiki corpus into a convokit Corpus object
data_dir = os.path.join(pkg_resources.resource_filename("convokit", ""), 'downloads')
corpus = Corpus(filename=os.path.join(data_dir, 'wiki-corpus'))

# if this is our first time running this example, we need to fit motifs on the
# dataset. If we have run it before, just load the previously computed motifs
motifs_dir = os.path.join(data_dir, dataset_name + "-motifs")
if not os.path.exists(motifs_dir):
    motifs_dir = None

questionTypology = QuestionTypology(corpus, data_dir, dataset_name=dataset_name, motifs_dir=motifs_dir,
                                    num_dims=50, num_clusters=6, question_threshold=100, answer_threshold=100, 
                                    verbose=1000000, random_seed=2018, min_support=20,
                                    questions_only=False, enforce_formatting=False,
                                    spacy_dir="/scratch/convokit_spacy_dump")

building q-a matrices
matrix dir /home/jonathan/research/Cornell-Conversational-Analysis-Toolkit/convokit/downloads/wiki-corpus-awry-matrix exists!
	reading arcs and motifs
	1000000
	2000000
	3000000
	4000000
	5000000
	6000000
	building matrices
	writing stuff
reading question tidxes
reading question leaves
reading answer tidxes
reading question didxes
reading answer didxes
reading question terms
reading answer terms
reading docs
done!


Now that the QuestionTypology object has been trained, we can use it to compute prompt types for our awry corpus (notice that this is a different corpus from what the QuestionTypology object was trained on!). For our purposes, we want the raw features, which are distances from the centers of the KMeans clusters corresponding to each prompt type. We can get these using the `get_qtype_dists` method. Note that in most other situations, where we want just the prompt type, we can use the QuestionTypology object as a Callable, which will return an integer prompt type for each utterance in the corpus. We could also use the `compute_type` function, which is aliased to do the same thing.

In [6]:
prompt_types = questionTypology.get_qtype_dists(awry_corpus)

In [7]:
# let's take a look at what the output looks like
prompt_types.head(10)

,km_0_dist,km_1_dist,km_2_dist,km_3_dist,km_4_dist,km_5_dist,content
146743638.12667.12652,0.988895,1.100313,0.925354,1.052824,0.998516,1.063603,I notice that earier that moved wiki_link to ...
146842219.12874.12874,0.934950,0.959767,0.906565,0.963544,0.903838,0.931161,"Chen was known in the poker world as ""William""..."
146860774.13072.13072,0.825559,1.048260,0.885811,0.764113,1.021051,0.985920,I see what you saying I just read his pokersta...
143890867.11944.11926,0.934407,0.973923,1.018610,1.163208,0.889388,1.110955,No more than two editors advocated deletion. ...
143902946.11991.11991,0.971398,0.712804,0.983160,0.916819,0.911771,0.839036,In the future please don't close Afds when you...
143945536.12065.12065,1.044573,0.976042,0.979892,1.099157,0.934318,1.022813,That simply isn't true. If you read the comme...
144052463.12169.12169,0.924036,1.102745,0.929785,0.959193,0.996333,0.963547,"Somehow, I suspect you may wish to participate..."
144065917.12226.12226,0.957274,0.687598,0.990034,0.996022,0.880861,0.844266,"I assume your deliberate lying has a point, bu..."
127296808.516.516,1.121714,0.692300,1.117303,1.166274,0.929166,1.001746,== Could you stop reverting my corrections ==\n
127296808.534.516,0.923464,0.784017,0.955954,0.741971,0.962447,0.760987,If you have problems with my edits to the 4WD ...


We're going to do a little bit of cleaning up of the prompt types table. First off, we don't need the content column, as we are just interested in using the prompt type features themselves. Second, in the paper we assigned a max distance cutoff, such that distances were capped at 1.

In [8]:
prompt_types = prompt_types.drop(columns="content")
prompt_types[prompt_types > 1] = 1

In [9]:
prompt_types.head(10)

,km_0_dist,km_1_dist,km_2_dist,km_3_dist,km_4_dist,km_5_dist
146743638.12667.12652,0.988895,1.000000,0.925354,1.000000,0.998516,1.000000
146842219.12874.12874,0.934950,0.959767,0.906565,0.963544,0.903838,0.931161
146860774.13072.13072,0.825559,1.000000,0.885811,0.764113,1.000000,0.985920
143890867.11944.11926,0.934407,0.973923,1.000000,1.000000,0.889388,1.000000
143902946.11991.11991,0.971398,0.712804,0.983160,0.916819,0.911771,0.839036
143945536.12065.12065,1.000000,0.976042,0.979892,1.000000,0.934318,1.000000
144052463.12169.12169,0.924036,1.000000,0.929785,0.959193,0.996333,0.963547
144065917.12226.12226,0.957274,0.687598,0.990034,0.996022,0.880861,0.844266
127296808.516.516,1.000000,0.692300,1.000000,1.000000,0.929166,1.000000
127296808.534.516,0.923464,0.784017,0.955954,0.741971,0.962447,0.760987


## Step 3: Extract politeness strategies features

Now we will extract the second type of pragmatic features described in the paper: politeness strategies. We can do this using convokit's PolitenessStrategies class. This class does not require any training, so we can just apply it directly to the corpus.

In [10]:
ps = PolitenessStrategies(awry_corpus)

The politeness strategy features themselves can be accessed via the `feature_df` field of the PolitenessStrategies object.

In [11]:
politeness_strategies = ps.feature_df
politeness_strategies.head(10)

,feature_politeness_==Hedges==,feature_politeness_==HASPOSITIVE==,feature_politeness_==Apologizing==,feature_politeness_==1st_person_pl.==,feature_politeness_==2nd_person==,feature_politeness_==Indirect_(btw)==,feature_politeness_==Indirect_(greeting)==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==HASHEDGE==,feature_politeness_==Please==,...,feature_politeness_==2nd_person_start==,feature_politeness_==Gratitude==,feature_politeness_==Direct_start==,feature_politeness_==Direct_question==,feature_politeness_==1st_person==,feature_politeness_==HASNEGATIVE==,feature_politeness_==1st_person_start==,feature_politeness_==INDICATIVE==,feature_politeness_==Deference==,feature_politeness_==Factuality==
146743638.12652.12652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146743638.12667.12652,1,1,0,0,1,0,0,0,1,0,...,0,0,0,1,1,1,1,0,0,0
146842219.12874.12874,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
146860774.13072.13072,1,1,0,0,1,0,0,0,1,0,...,0,1,1,0,1,1,1,0,0,0
143890867.11926.11926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143890867.11944.11926,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
143902946.11991.11991,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
143945536.12065.12065,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
144052463.12169.12169,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,0
144065917.12226.12226,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0


## Step 4: Create pair data

The prediction task defined in the paper is a paired task. The corpus downloaded from convokit already includes metadata about how conversations were paired for the paper, so we don't need to do any of the hard work here. Instead, we'll format the pair information into a table for use in prediction.

In [12]:
# first, we need to directly map comment IDs to their conversations. We'll build a DataFrame to do this
comment_ids = []
convo_ids = []
timestamps = []
page_ids = []
for comment_id in awry_corpus.utterances:
    comment = awry_corpus.utterances[comment_id]
    # section headers are included in the dataset for completeness, but for prediction we need to ignore
    # them as they are not utterances
    if not comment.other["is_section_header"]:
        comment_ids.append(comment_id)
        convo_ids.append(comment.root)
        timestamps.append(comment.timestamp)
        page_ids.append(comment.other["awry_info"]["page_id"])
comment_df = pd.DataFrame({"conversation_id": convo_ids, "timestamp": timestamps, "page_id": page_ids}, index=comment_ids)

In [13]:
# we'll do our construction using awry conversation ID's as the reference key
awry_convo_ids = set()
# these dicts will then all be keyed by awry ID
good_convo_map = {}
page_id_map = {}
for comment in awry_corpus.utterances.values():
    if comment.other["awry_info"]["conversation_has_personal_attack"] and comment.root not in awry_convo_ids:
        awry_convo_ids.add(comment.root)
        good_convo_map[comment.root] = comment.other["awry_info"]["pair_id"]
        page_id_map[comment.root] = comment.other["awry_info"]["page_id"]
awry_convo_ids = list(awry_convo_ids)
pairs_df = pd.DataFrame({"bad_conversation_id": awry_convo_ids,
                         "conversation_id": [good_convo_map[cid] for cid in awry_convo_ids],
                         "page_id": [page_id_map[cid] for cid in awry_convo_ids]})

In [14]:
# finally, we will augment the pairs dataframe with the IDs of the first and second comment for both
# the bad and good conversation. This will come in handy for constructing the feature matrix.
first_ids = []
second_ids = []
first_ids_bad = []
second_ids_bad = []
for row in pairs_df.itertuples():
    # "first two" is defined in terms of time of posting
    comments_sorted = comment_df[comment_df.conversation_id==row.conversation_id].sort_values(by="timestamp")
    first_ids.append(comments_sorted.iloc[0].name)
    second_ids.append(comments_sorted.iloc[1].name)
    comments_sorted_bad = comment_df[comment_df.conversation_id==row.bad_conversation_id].sort_values(by="timestamp")
    first_ids_bad.append(comments_sorted_bad.iloc[0].name)
    second_ids_bad.append(comments_sorted_bad.iloc[1].name)
pairs_df = pairs_df.assign(first_id=first_ids, second_id=second_ids, 
                           bad_first_id=first_ids_bad, bad_second_id=second_ids_bad)

## Step 5: Construct feature matrix

Now that we have the pair data, we can construct a table of pragmatic features for each pair, to use in prediction. This table will consist of the prompt types and politeness strategies for the first and second comment of each conversation.

In [15]:
def features_for_convo(convo_id, first_comment_id, second_comment_id):

    # get prompt type features
    try:
        first_prompts = prompt_types.loc[first_comment_id]
    except:
        first_prompts = pd.Series(data=np.ones(len(prompt_types.columns)), index=prompt_types.columns)
    try:
        second_prompts = prompt_types.loc[second_comment_id].rename({c: c + "_second" for c in prompt_types.columns})
    except:
        second_prompts = pd.Series(data=np.ones(len(prompt_types.columns)), index=[c + "_second" for c in prompt_types.columns])
    prompts = first_prompts.append(second_prompts)
    # get politeness strategies features
    first_politeness = politeness_strategies.loc[first_comment_id]
    second_politeness = politeness_strategies.loc[second_comment_id].rename({c: c + "_second" for c in politeness_strategies.columns})
    politeness = first_politeness.append(second_politeness)
    return politeness.append(prompts)

In [16]:
convo_ids = np.concatenate((pairs_df.conversation_id.values, pairs_df.bad_conversation_id.values))
feats = [features_for_convo(row.conversation_id, row.first_id, row.second_id) for row in pairs_df.itertuples()] + \
        [features_for_convo(row.bad_conversation_id, row.bad_first_id, row.bad_second_id) for row in pairs_df.itertuples()]
feature_table = pd.DataFrame(data=np.vstack([f.values for f in feats]), columns=feats[0].index, index=convo_ids)

In [17]:
# in the paper, we dropped the sentiment lexicon based features (HASPOSITIVE and HASNEGATIVE), opting
# to instead use them as a baseline. We do this here as well to be consistent with the paper.
feature_table = feature_table.drop(columns=["feature_politeness_==HASPOSITIVE==",
                                            "feature_politeness_==HASNEGATIVE==",
                                            "feature_politeness_==HASPOSITIVE==_second",
                                            "feature_politeness_==HASNEGATIVE==_second"])

In [18]:
# let's see how it looks
feature_table.head(5)

,feature_politeness_==Hedges==,feature_politeness_==Apologizing==,feature_politeness_==1st_person_pl.==,feature_politeness_==2nd_person==,feature_politeness_==Indirect_(btw)==,feature_politeness_==Indirect_(greeting)==,feature_politeness_==SUBJUNCTIVE==,feature_politeness_==HASHEDGE==,feature_politeness_==Please==,feature_politeness_==Please_start==,...,km_2_dist,km_3_dist,km_4_dist,km_5_dist,km_0_dist_second,km_1_dist_second,km_2_dist_second,km_3_dist_second,km_4_dist_second,km_5_dist_second
339153648.13082.13082,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.836569,0.799070,1.0,1.000000,0.983598,1.000000,0.822115,0.973038,0.997192,1.000000
273320587.1690.1690,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.0,1.000000,0.830685,1.000000,0.884913,0.804456,1.000000,1.000000
295446996.20932.20932,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.931499,0.770019,1.0,0.897080,1.000000,1.000000,1.000000,1.000000,0.975853,1.000000
114831022.4571.4571,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.809935,0.830955,1.0,1.000000,0.844493,1.000000,0.889421,0.816545,1.000000,0.989362
46490327.5727.5727,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.950241,0.722807,1.0,0.902583,0.883167,0.950272,0.931917,0.923855,0.910283,0.849097


## Step 6: Prediction Utils

We're almost ready to do the prediction! First we need to define a few helper functions...

In [19]:
def mode(seq):
    vals, counts = np.unique(seq, return_counts=True)
    return vals[np.argmax(counts)]

def run_pred_single(inputs, X, y):
    f_idx, (train_idx, test_idx) = inputs
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    base_clf = Pipeline([("scaler", StandardScaler()), ("featselect", SelectPercentile(f_classif, 10)), ("logreg", LogisticRegression())])
    clf = GridSearchCV(base_clf, {"logreg__C": [10**i for i in range(-4,4)], "featselect__percentile": list(range(10, 110, 10))})

    clf.fit(X_train, y_train)
    
    y_scores = clf.predict_proba(X_test)[:,1]
    y_pred = clf.predict(X_test)
    
    feature_weights = clf.best_estimator_.named_steps["logreg"].coef_.flatten()
    feature_mask = clf.best_estimator_.named_steps["featselect"].get_support()
    
    hyperparams = clf.best_params_
    
    return (y_pred, y_scores, feature_weights, hyperparams, feature_mask)

def run_pred(X, y, fnames, groups):
    feature_weights = {}
    scores = np.asarray([np.nan for i in range(len(y))])
    y_pred = np.zeros(len(y))
    hyperparameters = defaultdict(list)
    splits = list(enumerate(LeaveOneGroupOut().split(X, y, groups)))
    accs = []
        
    with Pool(os.cpu_count()) as p:
        prediction_results = p.map(partial(run_pred_single, X=X, y=y), splits)
        
    fselect_pvals_all = []
    for i in range(len(splits)):
        f_idx, (train_idx, test_idx) = splits[i]
        y_pred_i, y_scores_i, weights_i, hyperparams_i, mask_i = prediction_results[i]
        y_pred[test_idx] = y_pred_i
        scores[test_idx] = y_scores_i
        feature_weights[f_idx] = np.asarray([np.nan for _ in range(len(fnames))])
        feature_weights[f_idx][mask_i] = weights_i
        for param in hyperparams_i:
            hyperparameters[param].append(hyperparams_i[param])   
    
    acc = np.mean(y_pred == y)
    pvalue = stats.binom_test(sum(y_pred == y), n=len(y), alternative="greater")
                
    coef_df = pd.DataFrame(feature_weights, index=fnames)
    coef_df['mean_coef'] = coef_df.apply(np.nanmean, axis=1)
    coef_df['std_coef'] = coef_df.apply(np.nanstd, axis=1)
    return acc, coef_df[['mean_coef', 'std_coef']], scores, pd.DataFrame(hyperparameters), pvalue

def get_labeled_pairs(pairs_df):
    paired_labels = []
    c0s = []
    c1s = []
    page_ids = []
    for i, row in enumerate(pairs_df.itertuples()):
        if i % 2 == 0:
            c0s.append(row.conversation_id)
            c1s.append(row.bad_conversation_id)
        else:
            c0s.append(row.bad_conversation_id)
            c1s.append(row.conversation_id)
        paired_labels.append(i%2)
        page_ids.append(row.page_id)
    return pd.DataFrame({"c0": c0s, "c1": c1s,"first_convo_toxic": paired_labels, "page_id": page_ids})

def get_feature_subset(labeled_pairs_df, feature_list):
    prompt_type_names = ["km_%d_dist" % i for i in range(6)] + ["km_%d_dist_second" % i for i in range(6)]
    politeness_names = [f for f in feature_table.columns if f not in prompt_type_names]
    
    features_to_use = []
    if "prompt_types" in feature_list:
        features_to_use += prompt_type_names
    if "politeness_strategies" in feature_list:
        features_to_use += politeness_names
        
    feature_subset = feature_table[features_to_use]
    
    c0_feats = feature_subset.loc[labeled_pairs_df.c0].values
    c1_feats = feature_subset.loc[labeled_pairs_df.c1].values
    
    return c0_feats, c1_feats, features_to_use

def run_pipeline(feature_set):
    print("Running prediction task for feature set", "+".join(feature_set))
    print("Generating labels...")
    labeled_pairs_df = get_labeled_pairs(pairs_df)
    print("Computing paired features...")
    X_c0, X_c1, feature_names = get_feature_subset(labeled_pairs_df, feature_set)
    X = X_c1 - X_c0
    print("Using", X.shape[1], "features")
    y = labeled_pairs_df.first_convo_toxic.values
    print("Running leave-one-page-out prediction...")
    accuracy, coefs, scores, hyperparams, pvalue = run_pred(X, y, feature_names, labeled_pairs_df.page_id)
    print("Accuracy:", accuracy)
    print("p-value: %.4e" % pvalue)
    print("C (mode):", mode(hyperparams.logreg__C))
    print("Percent of features (mode):", mode(hyperparams.featselect__percentile))
    print("Coefficents:")
    print(coefs.sort_values(by="mean_coef"))
    return accuracy

## Step 7: Prediction

Finally, we run the prediction task on each possible combination of pragmatic features: prompt types, politeness strategies, and both combined. We generate a table like Table 3 from the paper.

In [20]:
feature_combos = [["politeness_strategies"], ["prompt_types"], ["politeness_strategies", "prompt_types"]]
combo_names = []
accs = []
for combo in feature_combos:
    combo_names.append("+".join(combo).replace("_", " "))
    accuracy = run_pipeline(combo)
    accs.append(accuracy)
results_df = pd.DataFrame({"Accuracy": accs}, index=combo_names)
results_df.index.name = "Feature set"

Running prediction task for feature set politeness_strategies
Generating labels...
Computing paired features...
Using 38 features
Running leave-one-page-out prediction...
Accuracy: 0.6078740157480315
p-value: 3.0260e-08
C (mode): 0.01
Percent of features (mode): 50
Coefficents:
                                                   mean_coef  std_coef
feature_politeness_==2nd_person==_second           -0.176311  0.043429
feature_politeness_==2nd_person_start==_second     -0.134761  0.031002
feature_politeness_==2nd_person_start==            -0.119161  0.029131
feature_politeness_==Direct_question==_second      -0.117183  0.028071
feature_politeness_==Direct_question==             -0.097743  0.020645
feature_politeness_==Please_start==_second         -0.090237  0.021035
feature_politeness_==Indirect_(btw)==              -0.086099  0.029164
feature_politeness_==Indirect_(btw)==_second       -0.040407  0.004392
feature_politeness_==Factuality==                  -0.036247  0.003115
feature_pol

/home/jonathan/miniconda3/envs/convokit/lib/python3.6/site-packages/pandas/core/apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)
/home/jonathan/miniconda3/envs/convokit/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Accuracy: 0.6251968503937008
p-value: 1.4664e-10
C (mode): 0.0001
Percent of features (mode): 50
Coefficents:
                                                   mean_coef      std_coef
feature_politeness_==2nd_person==_second           -0.021106  2.081326e-02
feature_politeness_==2nd_person_start==_second     -0.017606  1.707194e-02
feature_politeness_==Direct_question==_second      -0.015014  1.436886e-02
feature_politeness_==2nd_person_start==            -0.014947  1.401025e-02
km_2_dist                                          -0.014106  1.256456e-02
feature_politeness_==Direct_question==             -0.012878  1.135110e-02
feature_politeness_==Please_start==_second         -0.011870  1.067424e-02
km_2_dist_second                                   -0.011268  8.737590e-03
km_3_dist                                          -0.010420  8.864930e-03
km_3_dist_second                                   -0.009586  7.886899e-03
feature_politeness_==Indirect_(btw)==              -0.008070  1.0

In [21]:
# let's see the table
results_df

,Accuracy
Feature set,
politeness strategies,0.607874
prompt types,0.595276
politeness strategies+prompt types,0.625197
